# Finding and Imputing Missing Variables

This notebook will walk through several methods for imputing missing variables, including mean/median/mode imputation, imputation by linear regression, hot deck imputation, and case deletion. If there's time, I will also explore expectation maximization imputation and Markov Chain Monte Carlo algorithm

First, I need to:
- Read in the data, relevant libraries, and atom file for the code
- Find the missing data
- Determine if the data is missing completeley at random, missing at random, or missing not at random

In [128]:
# Importing code file
import missing_data_code as mdc
import importlib
importlib.reload(mdc)

<module 'missing_data_code' from '/Users/mariomoreno/Desktop/ML Projects/Mexico OC/missing_data_code.py'>

In [105]:
# Reading in data
df = mdc.reading_in('data/Mexico_Final.xlsx')

In [106]:
df.head()

,entidad,year,alumnos_inscritos_hs,hombres_inscritos_hs,mujeres_inscritas_hs,maestros_hs,escuelas_hs,convenios_trabajo_no_juicio,conflictos_trabajo,emplazamientos_huelga,...,prod_zinc_t,prod_coque_t,prod_fierropellets_t,prod_azufre_t,prod_barita_t,prod_fluorita_t,altitud,costa_km,municipios,pop
0,Aguascalientes,2010,40129,19250,20879,2727,144,2698,3239,325,...,0.0,0.0,3883423.0,0.0,0.0,0.0,1870,NaN,11,1195787
1,Baja California,2010,107624,53692,53932,6192,271,2342,9353,3241,...,0.0,0.0,3883423.0,0.0,0.0,0.0,3,1493.0,5,3224843
2,Baja California Sur,2010,23247,11780,11467,1588,76,1255,1660,296,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,2131.0,5,649616
3,Campeche,2010,28350,14349,14001,1975,106,3373,1179,144,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,425.0,11,836747
4,Coahuila de Zaragoza,2010,82553,41397,41156,6219,330,23331,7867,545,...,0.0,1648709.0,2567865.0,0.0,22161.0,121833.0,1700,NaN,38,2782012


In [107]:
# Finding the nulls
null_cols = mdc.find_missing_cols(df)
null_cols

[('prod_oro_kg', 7),
 ('prod_plata_kg', 7),
 ('prod_plomo_t', 7),
 ('prod_cobre_t', 7),
 ('prod_zinc_t', 7),
 ('prod_coque_t', 7),
 ('prod_fierropellets_t', 7),
 ('prod_azufre_t', 7),
 ('prod_barita_t', 7),
 ('prod_fluorita_t', 7),
 ('costa_km', 105)]

As we can see above, we're missing seven variables for each of the metal producing categories, and 105 variables for coastline. In order to determine if these variables are missing at random, I need to see what states correspond to the missing variables.

In [108]:
# Finding the row, column pairs that have missing values
missing_rows = mdc.find_missing_rows(df, 'entidad', null_cols)
missing_rows

{('Aguascalientes           ', 'costa_km'),
 ('Chihuahua                ', 'costa_km'),
 ('Ciudad de México b/', 'costa_km'),
 ('Coahuila de Zaragoza', 'costa_km'),
 ('Durango                  ', 'costa_km'),
 ('Guanajuato               ', 'costa_km'),
 ('Hidalgo                  ', 'costa_km'),
 ('Morelos                  ', 'costa_km'),
 ('México                   ', 'costa_km'),
 ('Nuevo León               ', 'costa_km'),
 ('Puebla                   ', 'costa_km'),
 ('Querétaro', 'costa_km'),
 ('Quintana Roo             ', 'prod_azufre_t'),
 ('Quintana Roo             ', 'prod_barita_t'),
 ('Quintana Roo             ', 'prod_cobre_t'),
 ('Quintana Roo             ', 'prod_coque_t'),
 ('Quintana Roo             ', 'prod_fierropellets_t'),
 ('Quintana Roo             ', 'prod_fluorita_t'),
 ('Quintana Roo             ', 'prod_oro_kg'),
 ('Quintana Roo             ', 'prod_plata_kg'),
 ('Quintana Roo             ', 'prod_plomo_t'),
 ('Quintana Roo             ', 'prod_zinc_t'),
 ('San 

From the set of states that have nulls for the variable costa_km, it seems that they're all land-locked states. While these values are missing at random, it'll be appropriate to impute them with a zero.

From the set of states that have nulls for metal production, the only one is Quintana Roo. It's unlikely that Quintana Roo does not produce any metals -- indeed, there are states with values of zero in those columns. As to why they're missing, it's unlikely that they're related to our ultimate aim of measuring presence of organized crime so we'll impute them using the methods below. 

But first, to impute missing coastline variables with zeroes.

### 1) Zero Imputation 

In [115]:
# Imputing zeroes for missing variables in costa_km column
mdc.impute_zero(df, {'costa_km':0})
df.isnull().sum()

entidad                               0
year                                  0
alumnos_inscritos_hs                  0
hombres_inscritos_hs                  0
mujeres_inscritas_hs                  0
maestros_hs                           0
escuelas_hs                           0
convenios_trabajo_no_juicio           0
conflictos_trabajo                    0
emplazamientos_huelga                 0
emplazamientos_huelga_solucionados    0
huelgas_estalladas                    0
huelgas_locales_solucionadas          0
denuncias_total                       0
denuncias_salud                       0
denuncias_arma                        0
denuncias_bancaria                    0
denuncias_leyg                        0
denuncias_comms                       0
denuncias_servpub                     0
denuncias_fiscales                    0
denuncias_ambiente                    0
denuncias_asocdelictuosa              0
denuncias_patrimoniales               0
denuncias_propintelectual             0


In [116]:
df.head()

,entidad,year,alumnos_inscritos_hs,hombres_inscritos_hs,mujeres_inscritas_hs,maestros_hs,escuelas_hs,convenios_trabajo_no_juicio,conflictos_trabajo,emplazamientos_huelga,...,prod_zinc_t,prod_coque_t,prod_fierropellets_t,prod_azufre_t,prod_barita_t,prod_fluorita_t,altitud,costa_km,municipios,pop
0,Aguascalientes,2010,40129,19250,20879,2727,144,2698,3239,325,...,0.0,0.0,3883423.0,0.0,0.0,0.0,1870,0.0,11,1195787
1,Baja California,2010,107624,53692,53932,6192,271,2342,9353,3241,...,0.0,0.0,3883423.0,0.0,0.0,0.0,3,1493.0,5,3224843
2,Baja California Sur,2010,23247,11780,11467,1588,76,1255,1660,296,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,2131.0,5,649616
3,Campeche,2010,28350,14349,14001,1975,106,3373,1179,144,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,425.0,11,836747
4,Coahuila de Zaragoza,2010,82553,41397,41156,6219,330,23331,7867,545,...,0.0,1648709.0,2567865.0,0.0,22161.0,121833.0,1700,0.0,38,2782012


The array and dataframe above shows that we've successfully imputed all missing values for coast_km with zeroes. Next, it's time to explore different imputation methods for the missing metals for Queretaro.

### 2) Case Deletion 

Just to track the shape of our database over time and make sure deletion is working, it's important to note that it's 224 rows and 46 columns. 

In [119]:
df.shape

(224, 46)

With that done, time to delete. We'll start with deleting the rows that have nulls (in this case, all the Quintana Roo entries). We should expect our row number (224) to decrease by seven to 217

In [130]:
deleted_rows = mdc.case_deletion(df, 'row')
deleted_rows.shape

(217, 46)

In [134]:
deleted_rows.head()

,entidad,year,alumnos_inscritos_hs,hombres_inscritos_hs,mujeres_inscritas_hs,maestros_hs,escuelas_hs,convenios_trabajo_no_juicio,conflictos_trabajo,emplazamientos_huelga,...,prod_zinc_t,prod_coque_t,prod_fierropellets_t,prod_azufre_t,prod_barita_t,prod_fluorita_t,altitud,costa_km,municipios,pop
0,Aguascalientes,2010,40129,19250,20879,2727,144,2698,3239,325,...,0.0,0.0,3883423.0,0.0,0.0,0.0,1870,0.0,11,1195787
1,Baja California,2010,107624,53692,53932,6192,271,2342,9353,3241,...,0.0,0.0,3883423.0,0.0,0.0,0.0,3,1493.0,5,3224843
2,Baja California Sur,2010,23247,11780,11467,1588,76,1255,1660,296,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,2131.0,5,649616
3,Campeche,2010,28350,14349,14001,1975,106,3373,1179,144,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10,425.0,11,836747
4,Coahuila de Zaragoza,2010,82553,41397,41156,6219,330,23331,7867,545,...,0.0,1648709.0,2567865.0,0.0,22161.0,121833.0,1700,0.0,38,2782012


Now we'll try deleting the columns that have nulls (in this case, all the metals). We should expect our column number (46) to decrease by ten to 36.

In [131]:
deleted_columns = mdc.case_deletion(df, 'column')
deleted_columns.shape

(224, 36)

In [133]:
deleted_columns.columns

Index(['entidad', 'year', 'alumnos_inscritos_hs', 'hombres_inscritos_hs',
       'mujeres_inscritas_hs', 'maestros_hs', 'escuelas_hs',
       'convenios_trabajo_no_juicio', 'conflictos_trabajo',
       'emplazamientos_huelga', 'emplazamientos_huelga_solucionados',
       'huelgas_estalladas', 'huelgas_locales_solucionadas', 'denuncias_total',
       'denuncias_salud', 'denuncias_arma', 'denuncias_bancaria',
       'denuncias_leyg', 'denuncias_comms', 'denuncias_servpub',
       'denuncias_fiscales', 'denuncias_ambiente', 'denuncias_asocdelictuosa',
       'denuncias_patrimoniales', 'denuncias_propintelectual',
       'denuncias_robocarretera', 'denuncias_otras', 'total_carretera_km',
       'carretera_pavimentada_km', 'carretera_revestida_km',
       'carretera_terraceria_kg', 'brechas_mejoradas_km', 'altitud',
       'costa_km', 'municipios', 'pop'],
      dtype='object')

Pretty straightforward. We now have two dataframes:
- deleted_rows is the dataframe with all the rows with missing values deleted
- deleted_columns in the dataframe with all the columns with missing values deleted

There is a rule of thumb in deleting rows or columns as a way to deal with missing values: **if a variable has more than 5% of it's values missing, do not delete**

While our current example meets the parameters of the rule of thumb, it's important to note that this is not the ideal way to deal with missing values, particularly in an index such as this.

That's why I'll now go through other forms of imputing missing variables. Starting with median imputation.

### 3) Median Imputation 

We'll go back to using our original dataframe -- df

In [ ]:
median_df = md